In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from tqdm import tqdm


### Parameters:

In [17]:
from transformers import AutoModel, AutoTokenizer


# model_name = "dbmdz/bert-base-italian-cased"
# XLM-RoBERTa is a multilingual version of RoBERTa
model_name = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): 

In [11]:
params = {
    'drop_prob': 0.1,
    'num_classes': 12
}

### Model:

In [10]:
class REModel_international(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.encoder = model
        hidden_size = 768    # Dimensionality of the encoder layers and the pooler layer.
        self.loss_fnt = nn.CrossEntropyLoss()
        self.classifier == nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(p = params['drop_prob']),
            nn.Linear(hidden_size, params['num_classes']),
        )
    
    def forward(self, input_ids = None, attention_mask = None, labels = None, ss = None, os = None):
        outputs = self.encoder(
            input_ids,
            attention_mask = attention_mask
        )
        pooled_output = outputs[0]
        idx = torch.arange(input_ids.size(0)).to(input_ids.device)
        # getting the embedded subject and object
        ss_emb = pooled_output[idx, ss]
        os_emb = pooled_output[idx, os]

        h = torch.cat((ss_emb, os_emb), dim = 1)
        # in classifier w/ Linear, Dropout and final Linear
        logits = self.classifier(h)
        outputs = (logits, )
        if labels is not None:
            loss = self.loss_fnt(logits.float(), labels)
            outputs = (loss, ) + outputs
        return outputs

SyntaxError: invalid syntax (1794868238.py, line 5)

In [22]:
## Just for trying the model:

sentence_t = tokenizer('Io sono Daniel', return_tensors="pt")
print(sentence_t)

m_output = model(
    sentence_t['input_ids'],
    attention_mask = sentence_t['attention_mask'],
)

m_output[0]

{'input_ids': tensor([[    0, 28136,  1389, 11555,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


tensor([[[ 8.3751e-02,  1.1660e-01,  4.2141e-02,  ..., -8.5256e-02,
           7.8344e-02,  1.9235e-03],
         [-1.8184e-02,  8.3488e-02,  3.2463e-03,  ...,  1.6405e-02,
          -1.9906e-02,  9.9005e-02],
         [ 2.1246e-02,  1.0508e-02, -5.1259e-02,  ..., -1.0199e-02,
          -4.7887e-02, -5.6288e-02],
         [ 2.4065e-02,  1.3772e-01, -2.2233e-02,  ..., -1.5960e-01,
           1.2269e-04,  2.4005e-01],
         [ 6.9008e-02,  1.0406e-01, -3.8184e-02,  ..., -2.0086e-01,
          -1.8710e-02,  5.8105e-02]]], grad_fn=<NativeLayerNormBackward0>)